In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb

In [3]:
import os
os.chdir("C:\\uit\\elliptic\\Elliptic_Dataset")
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("txs_features.csv")
print(df_txs_features)

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("txs_classes.csv")
print(df_txs_classes)

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("txs_edgelist.csv")

print(df_txs_edgelist)


Transaction features: 

             txId  Time step  Local_feature_1  Local_feature_2  \
0            3321          1        -0.169615        -0.184668   
1           11108          1        -0.137586        -0.184668   
2           51816          1        -0.170103        -0.184668   
3           68869          1        -0.114267        -0.184668   
4           89273          1         5.202107        -0.210553   
...           ...        ...              ...              ...   
203764  158304003         49        -0.165622        -0.139563   
203765  158303998         49        -0.167040        -0.139563   
203766  158303966         49        -0.167040        -0.139563   
203767  161526077         49        -0.172212        -0.139573   
203768  194103537         49        -0.172212        -0.139573   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
0             -1.201369        -0.121970        -0.043875        -0.113002   
1             -1.201369   

In [4]:
print("\ntxs_features.csv for txId = 272145560\n")
print(df_txs_features[df_txs_features['txId']==272145560])

print("\ntxs_classes.csv for txId = 272145560\n")
print(df_txs_classes[df_txs_classes['txId']==272145560])

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId = 272145560

             txId  Time step  Local_feature_1  Local_feature_2  \
105573  272145560         24        -0.155493        -0.107012   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
105573        -1.201369         -0.12197        -0.043875        -0.113002   

        Local_feature_7  Local_feature_8  ...  in_BTC_min  in_BTC_max  \
105573        -0.061584        -0.145749  ...      2.7732      2.7732   

        in_BTC_mean  in_BTC_median  in_BTC_total  out_BTC_min  out_BTC_max  \
105573       2.7732         2.7732        2.7732     0.001917     2.770883   

        out_BTC_mean  out_BTC_median  out_BTC_total  
105573        1.3864          1.3864         2.7728  

[1 rows x 184 columns]

txs_classes.csv for txId = 272145560

             txId  class
105573  272145560      1

txs_edgelist.csv for txId = 272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560


In [5]:
df_txs_features = df_txs_features.merge(df_txs_classes, left_on = "txId" , right_on = "txId" , how = "left")
# print(txs.shape)

print(df_txs_features["class"].value_counts())

class
3    157205
2     42019
1      4545
Name: count, dtype: int64


In [6]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [7]:
for column in df_txs_features.columns[168:]:
    if column != "class":
        feature = np.array(df_txs_features[column]).reshape(-1,1)
        scaler = MinMaxScaler()
        scaler.fit(feature)
        feature_scaled = scaler.transform(feature)
        df_txs_features[column] = feature_scaled.reshape(1,-1)[0]

In [8]:
# remove 'unknown' transactions
data = df_txs_features["txId"] #.loc[(df_txs_features['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features.loc[df_txs_features['txId'].isin(data)]
print(df_txs_features_selected["class"].value_counts())

class
3    156759
2     41500
1      4545
Name: count, dtype: int64


In [9]:
df_txs_features_selected

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,0.009891,0.009891,0.009891,0.009891,1.505606e-04,0.009935,0.018790,1.879015e-02,0.009891,3


In [10]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

X_data = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_training_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data)]
X_train = X_training_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35), 'txId']
X_testing_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test)]
X_test = X_testing_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test_class01 = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_testing_timesteps_class01 = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test_class01)]
X_test_class01 = X_testing_timesteps_class01.drop(columns=['txId', 'class', 'Time step'])

y_training_timesteps = X_training_timesteps[['class']]
y_training_timesteps = y_training_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_train = y_training_timesteps

y_testing_timesteps = X_testing_timesteps[['class']]
y_testing_timesteps = y_testing_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test = y_testing_timesteps

y_testing_timesteps_class01 = X_testing_timesteps_class01[['class']]
y_testing_timesteps_class01 = y_testing_timesteps_class01['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test_class01 = y_testing_timesteps_class01

In [11]:
y_train.value_counts()

class
1    26237
0     3462
Name: count, dtype: int64

In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train, y_train)


In [13]:
import torch
from torch_geometric.data import Data

# =========================
# 1. Chia node theo train/test (giữ lại txId)
# =========================
train_nodes = df_txs_features_selected[
    (df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3)
].copy()

test_nodes = df_txs_features_selected[
    (df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3)
].copy()
test_nodes_for_align = df_txs_features_selected[
    (df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3)
][['txId']].copy()
txids_test_order = test_nodes_for_align['txId'].astype(int).values
# Nhãn: map class=2 -> 0, class=1 -> 1
train_nodes['label'] = train_nodes['class'].apply(lambda x: x-1)
test_nodes['label']  = test_nodes['class'].apply(lambda x: x-1)

# =========================
# 2. Chuẩn bị feature tensor (drop txId, class, Time step)
# =========================
feature_cols = [c for c in df_txs_features_selected.columns if c not in ['txId','class','Time step']]

X_train_gcn = torch.tensor(train_nodes[feature_cols].values, dtype=torch.float)
y_train_gcn = torch.tensor(train_nodes['label'].values, dtype=torch.long)

X_test_gcn = torch.tensor(test_nodes[feature_cols].values, dtype=torch.float)
y_test_gcn = torch.tensor(test_nodes['label'].values, dtype=torch.long)

# =========================
# 3. Build edge_index cho train/test
# =========================
def build_edge_index(df_edges, valid_nodes):
    """Lọc edgelist chỉ chứa cạnh có cả 2 đầu nằm trong valid_nodes"""
    node_ids = valid_nodes['txId'].values
    id2idx = {tid: i for i, tid in enumerate(node_ids)}

    edges = df_edges[
        (df_edges['txId1'].isin(node_ids)) & (df_edges['txId2'].isin(node_ids))
    ].copy()

    if len(edges) > 0:
        edge_index = torch.tensor(
            edges[['txId1','txId2']].replace(id2idx).values.T,
            dtype=torch.long
        )
    else:
        edge_index = torch.empty((2,0), dtype=torch.long)

    return edge_index, id2idx

edge_index_train, id2idx_train = build_edge_index(df_txs_edgelist, train_nodes)
edge_index_test,  id2idx_test  = build_edge_index(df_txs_edgelist, test_nodes)

# =========================
# 4. Tạo PyG Data object
# =========================
# ngay sau khi tạo train_nodes / test_nodes và trước khi tạo Data(...)
train_txids = torch.tensor(train_nodes['txId'].astype(int).values, dtype=torch.long)
test_txids  = torch.tensor(test_nodes['txId'].astype(int).values,  dtype=torch.long)

train_data = Data(x=X_train_gcn, edge_index=edge_index_train, y=y_train_gcn)
test_data  = Data(x=X_test_gcn,  edge_index=edge_index_test,  y=y_test_gcn)


# lưu txId theo đúng thứ tự node
train_data.node_ids = train_txids
test_data.node_ids  = test_txids


In [14]:
print(train_data)
print(test_data)

Data(x=[29699, 182], edge_index=[2, 22796], y=[29699], node_ids=[29699])
Data(x=[16346, 182], edge_index=[2, 13616], y=[16346], node_ids=[16346])


In [15]:
print("Check NaN in train features:", torch.isnan(train_data.x).any().item())
print("Check Inf in train features:", torch.isinf(train_data.x).any().item())
print("Check NaN in train labels:", torch.isnan(train_data.y.float()).any().item())

Check NaN in train features: False
Check Inf in train features: False
Check NaN in train labels: False


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.metrics import precision_recall_fscore_support, f1_score, confusion_matrix

# ======================
# 1. Định nghĩa GCN model
# ======================
class GCN(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hid_dim)
        self.conv2 = GCNConv(hid_dim, out_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [17]:
from torch_geometric.nn import SAGEConv

class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hid_dim)
        self.conv2 = SAGEConv(hid_dim, out_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [19]:
# y_train là pandas Series → convert sang numpy
y_train_np = np.array(y_train)

# Đếm số mẫu của mỗi lớp
unique, counts = np.unique(y_train_np, return_counts=True)
print("Class distribution:", dict(zip(unique, counts)))

# Nếu thiếu lớp nào thì bổ sung 0
count_0 = counts[unique.tolist().index(0)] if 0 in unique else 0
count_1 = counts[unique.tolist().index(1)] if 1 in unique else 0
class_sample_count = torch.tensor([count_0, count_1], dtype=torch.float)

# Tránh chia 0
eps = 1e-8
weights = 1.0 / (class_sample_count + eps)

Class distribution: {np.int64(0): np.int64(3462), np.int64(1): np.int64(26237)}


In [20]:
from torch.optim.lr_scheduler import StepLR
in_dim = train_data.x.size(1)
model = GraphSAGE(in_dim=in_dim, hid_dim=64, out_dim=2).to(device)

opt = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
# scheduler = StepLR(opt, step_size=100, gamma=0.5)
crit = nn.CrossEntropyLoss(weight=weights.to(device))

# ======================
# 3. Training loop
# ======================
def train_epoch(data):
    model.train()
    opt.zero_grad()
    out = model(data.x.to(device), data.edge_index.to(device))
    loss = crit(out, data.y.to(device))
    loss.backward()
    opt.step()
    return loss.item()

for epoch in range(1, 801):
    loss = train_epoch(train_data)
    if epoch % 50 == 0:
        print(f"[Epoch {epoch}] loss={loss:.4f}")
    # scheduler.step()



[Epoch 50] loss=0.1847
[Epoch 100] loss=0.1217
[Epoch 150] loss=0.1176
[Epoch 200] loss=0.1040
[Epoch 250] loss=0.1055
[Epoch 300] loss=0.1165
[Epoch 350] loss=0.1151
[Epoch 400] loss=0.1919
[Epoch 450] loss=0.1217
[Epoch 500] loss=0.2336
[Epoch 550] loss=0.1133
[Epoch 600] loss=0.1224
[Epoch 650] loss=0.1144
[Epoch 700] loss=0.1153
[Epoch 750] loss=0.1120
[Epoch 800] loss=0.1293


In [21]:
# ======================
# 4. Evaluation
# ======================
@torch.no_grad()
def evaluate(data, name="TEST"):
    model.eval()
    out = model(data.x.to(device), data.edge_index.to(device))
    preds = out.argmax(dim=1).cpu().numpy()
    y_true = data.y.cpu().numpy()

    prec, rec, f1, _ = precision_recall_fscore_support(y_true, preds, zero_division=0)
    micro_f1 = f1_score(y_true, preds, average='micro', zero_division=0)
    cm = confusion_matrix(y_true, preds, labels=[0,1])

    print(f"\n{name} Evaluation")
    print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (prec[0], rec[0], f1[0]))
    print("Micro-F1: %.3f" % micro_f1)
    print("Confusion Matrix:\n", cm)

# Evaluate cả train và test
evaluate(train_data, "TRAIN")
evaluate(test_data, "TEST")


TRAIN Evaluation
Precision: 0.798 
Recall: 0.976 
F1 Score: 0.878
Micro-F1: 0.968
Confusion Matrix:
 [[ 3380    82]
 [  856 25381]]

TEST Evaluation
Precision: 0.682 
Recall: 0.674 
F1 Score: 0.678
Micro-F1: 0.958
Confusion Matrix:
 [[  730   353]
 [  340 14923]]
